In [1]:
import cv2
import numpy as np

In [6]:
ann =cv2.ml.ANN_MLP_create()
ann.setLayerSizes(np.array([9,5,9],dtype=np.uint8))
ann.setTrainMethod(cv2.ml.ANN_MLP_BACKPROP)

ann.train(np.array([[1.2,1.3,1.9,2.2,2.3,2.9,3.0,3.2,3.3]],dtype=np.float32),
          cv2.ml.ROW_SAMPLE,np.array([[0,0,0,0,0,1,0,0,0]],dtype=np.float32))
print(ann.predict(np.array([[1.4,1.5,1.2,2.0,2.5,2.8,3.0,3.1,3.8]],dtype=np.float32)))


(5.0, array([[-0.06419383, -0.13360272, -0.1681568 , -0.18708915,  0.0970564 ,
         0.89237726,  0.05093023,  0.17537238,  0.13388439]], dtype=float32))


In [7]:
import cv2
import numpy as np
from random import randint

In [20]:
animals_net = cv2.ml.ANN_MLP_create()
animals_net.setTrainMethod(cv2.ml.ANN_MLP_BACKPROP | cv2.ml.ANN_MLP_UPDATE_WEIGHTS)
animals_net.setActivationFunction(cv2.ml.ANN_MLP_SIGMOID_SYM)
animals_net.setLayerSizes(np.array([3,8,4]))
animals_net.setTermCriteria((cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1))

def dog_sample():
    return [randint(5,20),1,randint(38,42)]

def dog_class():
    return [1,0,0,0]

def condor_sample():
    return [randint(3,13),3,0]

def condor_class():
    return [0,1,0,0]

def dolphin_sample():
    return [randint(30,190),randint(5,15),randint(80,100)]

def dolphin_class():
    return [0,0,1,0]

def dragon_sample():
    return [randint(1200,1800),randint(15,40),randint(110,180)]

def dragon_class():
    return [0,0,0,1]

SAMPLES = 5000
for x in range(0,SAMPLES):
#     print('samplees %d/%d' %(x,SAMPLES))
    animals_net.train(np.array([dog_sample()],dtype=np.float32),cv2.ml.ROW_SAMPLE,
                      np.array([dog_class()],dtype=np.float32))
    animals_net.train(np.array([condor_sample()],dtype=np.float32),cv2.ml.ROW_SAMPLE,
                     np.array([condor_class()],dtype=np.float32))
    animals_net.train(np.array([dragon_sample()],dtype=np.float32),cv2.ml.ROW_SAMPLE,
                     np.array([dragon_class()],dtype=np.float32))

print(animals_net.predict(np.array([dog_sample()],dtype=np.float32)))
print(animals_net.predict(np.array([condor_sample()],dtype=np.float32)))
print(animals_net.predict(np.array([dragon_sample()],dtype=np.float32)))

(0.0, array([[ 1.69311392, -0.5505088 ,  0.20958848, -0.27631417]], dtype=float32))
(0.0, array([[ 1.69153309, -0.54443794,  0.26567742, -0.26928836]], dtype=float32))
(3.0, array([[-1.69451082,  0.55339879,  0.51075745,  2.21038175]], dtype=float32))


In [27]:
dog_result = 0
condor_result= 0
for x in range(0,100):
    dog = int(animals_net.predict(np.array([dog_sample()],dtype=np.float32))[0])
#     print('class :%d' %dog)
    if (dog) == 0:
        dog_result += 1
    condor = int(animals_net.predict(np.array([condor_sample()],dtype=np.float32))[0])
#     print('class :%d' %condo)
    if (condo) == 0:
        condor_result += 1
        
        
print('Dog accuracy :%f' %(dog_result))
print('condor accuracy : %f' %(condor_result))

Dog accuracy :100.000000
condor accuracy : 100.000000


In [30]:
import cv2

import pickle

import numpy as np

import gzip


def load_data():

    mnist = gzip.open('./data/mnist.pkl.gz', 'rb')

    training_data, classification_data, test_data = pickle.load(mnist,encoding='latin1')

    mnist.close()

    return (training_data, classification_data, test_data)


def wrap_data():

    tr_d, va_d, te_d = load_data()

    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]

    training_results = [vectorized_result(y) for y in tr_d[1]]

    training_data = zip(training_inputs, training_results)

    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
 
    validation_data = zip(validation_inputs,va_d[1])

    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]

    test_data = zip(test_inputs, te_d[1])

    return (training_data, validation_data, test_data)

 

 

 # 给出类标签,创建10个元素的0数组

# 参数j表示要置1的位置

def vectorized_result(j):

    e= np.zeros((10, 1))

    e[j] = 1.0

    return e

 

 

# 创建ann

def create_ANN(hidden=20):

    ann = cv2.ml.ANN_MLP_create()

    #设置各层大小

    ann.setLayerSizes(np.array([784, hidden, 10]))

    #采用反向传播方式

    ann.setTrainMethod(cv2.ml.ANN_MLP_RPROP)

    ann.setActivationFunction(cv2.ml.ANN_MLP_SIGMOID_SYM)

    #指定ann的终止条件

    ann.setTermCriteria((cv2.TERM_CRITERIA_EPS | cv2.TermCriteria_COUNT, 20,1))

    return ann

 

 

# 训练函数

def train(ann, samples=10000, epochs=1):
 
    tr, val, test = wrap_data()

    for x in range(epochs):

        counter = 0

        for img in tr:

            if (counter > samples):
 
                 break

            if (counter % 1000 == 0):

                 print("Epoch %d: Trained%d/%d " % (x, counter, samples))

            counter += 1

            data, digit = img

           # ravel()将多维数组拉平为一维

            ann.train(np.array([data.ravel()], dtype=np.float32),
                      cv2.ml.ROW_SAMPLE,

                      np.array([digit.ravel()],dtype=np.float32))

        print('Epoch %d complete' % x)

    return ann, test

 

 

# 检查神精网络工作

def test(ann, test_data):

    sample = np.array(test_data[0][0].ravel(), dtype=np.float32).reshape(28,28)

    cv2.imshow("sample", sample)

    cv2.waitKey()

    print(ann.predict(np.array([test_data[0][0].ravel()],dtype=np.float32)))

 

 

def predict(ann, sample):

    resized = sample.copy()

    rows, cols = resized.shape

    if (rows != 28 or cols != 28) and rows * cols > 0:

        resized = cv2.resize(resized, (28, 28), interpolation=cv2.INTER_CUBIC)

    return ann.predict(np.array([resized.ravel()], dtype=np.float32))



if __name__ == "__main__":
    print(vectorized_result(2))
    


[[ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
